# Notebook 07: Proposal Order Amounts

This notebook generates sales orders for the 2022H3 proposal using forecasting and regression predictions generated in [Notebook 4: Forecasting Monthly](04_Forecasting_Monthly.ipynb) and [Notebook 5: Regression Predict Sales](05_Regression_Predict_Sales.ipynb).

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.metrics import mean_squared_error
    
from custom_functions import *
from regression_custom_functions import *

In [18]:
# Import datasets and predictions
df_proposal = pd.read_csv('../data/cleaned_datasets/df_models.csv')
proposal_predictions = pd.read_csv('../data/predictions/proposal_forecast.csv')
new_model_predictions = pd.read_csv('../data/predictions/new_models_predictions.csv')
new_model_classifications = pd.read_csv('../data/predictions/new_models_classifications.csv')

# Drop items not on proposal
df_proposal.dropna(subset='proposal_priority',inplace=True)

In [19]:
return_nondate_col(df_proposal).columns

Index(['style_id', 'max_cart_qty', 'availability', 'qty_on_hand',
       'qty_on_order', 'qty_total_inv', 'qty_sold_last_6m', 'qty_avg/mo',
       'months_of_supply', 'xyz_cost', 'qty_xyz_cost', 'wholesale_price',
       'country_of_origin', 'warranty', 'band_color', 'band_material',
       'case_material', 'dial_color', 'case_size', 'gender', 'color',
       'retail_price', 'collection', 'clasp_type', 'water_resistance',
       'movement_type', 'proposal_priority', 'status', 'planned_order'],
      dtype='object')

In [20]:
order_sheet_columns = ['style_id','proposal_priority', 'availability', 
       'max_cart_qty', 'retail_price', 'wholesale_price','xyz_cost','qty_total_inv', 'planned_order']

In [21]:
order_sheet = df_proposal[order_sheet_columns]

## Create new dataframe that merges proposal, proposal forecast, new models, and new model predictions to create an overall order.

In [22]:
order_sheet = order_sheet.merge(proposal_predictions, how = 'left', on = 'style_id')
order_sheet = order_sheet.merge(new_model_predictions, how = 'left', on = 'style_id')
order_sheet = order_sheet.merge(new_model_classifications, how = 'left', on = 'style_id')

In [23]:
order_sheet['quarterly_forecast'] = 0
order_sheet['order'] = 0

order_per_class = 10 # How many orders per class
for i in order_sheet.index:
    base = 5
    if str(order_sheet.loc[i,'monthly_forecast_Q3']) == "nan":
        quarterly = 3 * order_per_class * order_sheet.loc[i,'monthly_classifications_Q3']
        order = quarterly
    else:
        quarterly = 3 * order_sheet.loc[i,'monthly_forecast_Q3']
        order = quarterly - order_sheet.loc[i,'qty_total_inv']
        if order < 0:
            order = 0
    order_sheet.loc[i,'quarterly_forecast'] = round(quarterly)
    order_sheet.loc[i,'order'] = base * round(order/base)

In [24]:
# order_sheet['quarterly_forecast'] = 0
# order_sheet['order'] = 0
# for i in order_sheet.index:
#     base = 5
#     if str(order_sheet.loc[i,'monthly_forecast_Q3']) == "nan":
#         quarterly = 3 * order_sheet.loc[i,'monthly_predictions_Q3']
#         order = quarterly
#     else:
#         quarterly = 3 * order_sheet.loc[i,'monthly_forecast_Q3']
#         order = quarterly - order_sheet.loc[i,'qty_total_inv']
#         if order < 0:
#             order = 0
#     order_sheet.loc[i,'quarterly_forecast'] = round(quarterly)
#     order_sheet.loc[i,'order'] = base * round(order/base)

In [25]:
order_sheet

,style_id,proposal_priority,availability,max_cart_qty,retail_price,wholesale_price,xyz_cost,qty_total_inv,planned_order,monthly_forecast_Q3,monthly_predictions_Q3,monthly_classifications_Q3,quarterly_forecast,order
0,ce1107,c,available,100,219.0,109.5,76.65,12.0,10.0,5.537037,NaN,NaN,17,5
1,ce1108,c,available,100,219.0,109.5,76.65,17.0,0.0,2.259259,NaN,NaN,7,0
2,ce1111,c,available,100,230.0,115.0,80.50,0.0,10.0,0.000000,NaN,NaN,0,0
3,ce1112,c,available,100,219.0,109.5,76.65,8.0,5.0,3.666667,NaN,NaN,11,5
4,ce1113,c,available,100,219.0,109.5,76.65,14.0,5.0,4.333333,NaN,NaN,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,fs5949,2022h3,delivery more than 6 weeks,NaN,155.0,77.5,54.25,0.0,50.0,NaN,0.504753,0.250000,8,10
296,fs5950,2022h3,delivery more than 6 weeks,NaN,155.0,77.5,54.25,0.0,40.0,NaN,2.567843,0.250000,8,10
297,fs5951,2022h3,delivery more than 6 weeks,NaN,155.0,77.5,54.25,0.0,50.0,NaN,1.647060,0.250000,8,10
298,fs5952,2022h3,delivery more than 6 weeks,NaN,155.0,77.5,54.25,0.0,50.0,NaN,0.998357,0.250000,8,10


In [ ]:
plot_preds(order_sheet['planned_order'], order_sheet['order'], 'Order Estimate')

In [ ]:
mean_squared_error(order_sheet['planned_order'],order_sheet['order'],squared=False)

In [ ]:
order_sheet.to_csv('../data/predictions/order_sheet.csv',index=False)